In [ ]:
import numpy as np
import random

In [ ]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def sigmoid_derivative(A):
    return A * (1 - A)

In [ ]:
class NeuralNetwork:
    def __init__(self, X, Y, layer_sizes):
        self.X = X  # shape (m, n_features)
        self.Y = Y.reshape(1, -1)  # shape (1, m)
        self.layer_sizes = layer_sizes  # [n_input, n1, n2, ..., n_output]
        self.L = len(layer_sizes) - 1  # number of layers (excluding input)

        self.weights = {}
        self.biases = {}
        self.cache = {}

        # Weight initialization
        for l in range(1, len(layer_sizes)):
            self.weights[l] = np.random.randn(layer_sizes[l], layer_sizes[l - 1]) * 0.01
            self.biases[l] = np.zeros((layer_sizes[l], 1))

    def forward_propagation(self):
        A = self.X.T  # shape (n_features, m)
        self.cache['A0'] = A

        for l in range(1, self.L + 1):
            Z = self.weights[l] @ A + self.biases[l]
            A = sigmoid(Z)
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A

        return A  # output layer activation

    def compute_cost(self, AL):
        m = self.Y.shape[1]
        cost = -1 / m * np.sum(self.Y * np.log(AL) + (1 - self.Y) * np.log(1 - AL))
        return np.squeeze(cost)

    def backward_propagation(self):
        grads = {}
        m = self.X.shape[0]
        Y = self.Y
        L = self.L

        AL = self.cache[f'A{L}']
        dAL = AL - Y  # derivative of binary cross-entropy loss

        for l in reversed(range(1, L + 1)):
            A_prev = self.cache[f'A{l - 1}']
            Z = self.cache[f'Z{l}']
            dZ = dAL * sigmoid_derivative(AL) if l == L else grads[f'dA{l}'] * sigmoid_derivative(self.cache[f'A{l}'])
            grads[f'dW{l}'] = 1 / m * dZ @ A_prev.T
            grads[f'db{l}'] = 1 / m * np.sum(dZ, axis=1, keepdims=True)
            grads[f'dA{l - 1}'] = self.weights[l].T @ dZ
            dAL = grads[f'dA{l - 1}']

        return grads

    def update_parameters(self, grads, learning_rate):
        for l in range(1, self.L + 1):
            self.weights[l] -= learning_rate * grads[f'dW{l}']
            self.biases[l] -= learning_rate * grads[f'db{l}']

    def fit(self, iterations=1000, learning_rate=0.1, verbose=True):
        for i in range(iterations):
            AL = self.forward_propagation()
            cost = self.compute_cost(AL)
            grads = self.backward_propagation()
            self.update_parameters(grads, learning_rate)

            if verbose and i % 100 == 0:
                print(f"Iteration {i} - Cost: {cost:.4f}")

    def predict(self, X):
        A = X.T
        for l in range(1, self.L + 1):
            Z = self.weights[l] @ A + self.biases[l]
            A = sigmoid(Z)
        return (A > 0.5).astype(int)

    def accuracy(self, X, Y):
        predictions = self.predict(X)
        return np.mean(predictions.flatten() == Y.flatten())


In [ ]:
# Données de test (ex: XOR ou toute donnée binaire)
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
Y = np.array([[0],
              [1],
              [1],
              [0]])

# Création et entraînement du modèle
nn = NeuralNetwork(X, Y, layer_sizes=[2, 4, 1])
nn.fit(iterations=10000, learning_rate=0.5)

# Prédictions
print("Prédictions :", nn.predict(X).T)
print("Accuracy :", nn.accuracy(X, Y))